# Converting masked-based implementation to cropping-based implementation

In [0]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [0]:
test_ones_2d = np.ones([1, 5, 5, 1], dtype='float32')
test_ones_3d = np.ones([1, 5, 5, 5, 1], dtype='float32')

In [0]:
def print_test_3d(test_matrix):
    for i in range(test_matrix.shape[1]):
        print(f'Depth {i}')
        print(test_matrix[0,i,:,:,0])

In [4]:
print_test_3d(test_ones_3d)

Depth 0
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Depth 1
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Depth 2
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Depth 3
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
Depth 4
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


In [5]:
print(test_ones_2d[0,:,:,0].squeeze())

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]


## Creating 2D masked solution to check results with cropped solution later

In [0]:
class MaskedConv2D(tf.keras.layers.Layer):
    def __init__(self,
                 mask_type,
                 filters,
                 kernel_size,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros'):
        super(MaskedConv2D, self).__init__()

        assert mask_type in {'A', 'B', 'V'}
        self.mask_type = mask_type

        self.filters = filters

        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)
        self.kernel_size = kernel_size

        self.strides = strides
        self.padding = padding.upper()
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)

    def build(self, input_shape):
        kernel_h, kernel_w = self.kernel_size

        self.kernel = self.add_weight("kernel",
                                      shape=(kernel_h,
                                             kernel_w,
                                             int(input_shape[-1]),
                                             self.filters),
                                      initializer=self.kernel_initializer,
                                      trainable=True)

        self.bias = self.add_weight("bias",
                                    shape=(self.filters,),
                                    initializer=self.bias_initializer,
                                    trainable=True)

        mask = np.ones(self.kernel.shape, dtype=np.float32)

        if kernel_h % 2 != 0: 
            center_h = kernel_h // 2
        else:
            center_h = (kernel_h - 1) // 2

        if kernel_w % 2 != 0: 
            center_w = kernel_w // 2
        else:
            center_w = (kernel_w - 1) // 2



        if self.mask_type == 'V':
            mask[center_h + 1:, :, :, :] = 0.
        else:
            mask[center_h, center_w + (self.mask_type == 'B'):, :, :] = 0.
            mask[center_h + 1:, :, :] = 0.

        self.mask = tf.constant(mask, dtype=tf.float32, name='mask')

    def call(self, input):
        masked_kernel = tf.math.multiply(self.mask, self.kernel)
        x = tf.nn.conv2d(input, masked_kernel, strides=[1, self.strides, self.strides, 1], padding=self.padding)
        x = tf.nn.bias_add(x, self.bias)
        return x

In [0]:
def build_test_stack_2d(mask_type, input_shape=(5, 5, 1), kernel_size=(3, 3)):
  inputs = tf.keras.layers.Input(shape=input_shape)
  
  x = MaskedConv2D(mask_type=mask_type,
                   filters=1,
                   kernel_size=kernel_size, 
                   padding='same',
                   kernel_initializer='ones', 
                   bias_initializer='zeros')(inputs)

  stack = tf.keras.Model(inputs=inputs, outputs=x)
  stack.compile(optimizer='adam', loss='mse')
  return stack

### Tests with kernel_size 3

#### Vertical stack

In [93]:
mask_type = 'V'
kernel_size=(3, 3)

padding = keras.layers.ZeroPadding2D(padding=((1,0),0))

conv = MaskedConv2D(mask_type=mask_type,
                filters=1,
                kernel_size=kernel_size, 
                padding='same',
                kernel_initializer='ones', 
                bias_initializer='zeros')

cropping = keras.layers.Cropping2D(cropping=((0, 1), 0))


x = padding(test_ones_2d)
x = conv(x)
result = cropping(x)

print(result.numpy().squeeze())


[[0. 0. 0. 0. 0.]
 [2. 3. 3. 3. 2.]
 [4. 6. 6. 6. 4.]
 [4. 6. 6. 6. 4.]
 [4. 6. 6. 6. 4.]]


In [9]:
horizontal_A_stack = build_test_stack_2d('A', kernel_size=(1, 3))
val = horizontal_A_stack.predict(test_ones_2d)
print(val[0,:,:,0].squeeze())

[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]


In [10]:
horizontal_B_stack = build_test_stack_2d('B', kernel_size=(1, 3))
val = horizontal_B_stack.predict(test_ones_2d)
print(val[0,:,:,0].squeeze())

[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]


### Tests with kernel_size 4

In [11]:
vertical_stack = build_test_stack_2d('V', kernel_size=(4, 4))
val = vertical_stack.predict(test_ones_2d)
print(val[0,:,:,0].squeeze())

[[0. 0. 0. 0. 0.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]]


In [12]:
horizontal_A_stack = build_test_stack_2d('A', kernel_size=(1, 4))
val = horizontal_A_stack.predict(test_ones_2d)
print(val[0,:,:,0].squeeze())

[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]


In [13]:
horizontal_B_stack = build_test_stack_2d('B', kernel_size=(1, 4))
val = horizontal_B_stack.predict(test_ones_2d)
print(val[0,:,:,0].squeeze())

[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]


## Creating 3D masked solution to check results with cropped solution later

In [0]:
class MaskedConv3D(tf.keras.layers.Layer):
    def __init__(self,
                 mask_type,
                 filters,
                 kernel_size,
                 strides=1,
                 padding='same',
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros'):
        super(MaskedConv3D, self).__init__()

        assert mask_type in {'A', 'B', 'D', 'V'}
        self.mask_type = mask_type

        self.filters = filters

        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size, kernel_size)
        self.kernel_size = kernel_size

        self.strides = strides
        self.padding = padding.upper()
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)

    def build(self, input_shape):
        kernel_d, kernel_h, kernel_w = self.kernel_size

        self.kernel = self.add_weight("kernel",
                                      shape=(kernel_d,
                                             kernel_h,
                                             kernel_w,
                                             int(input_shape[-1]),
                                             self.filters),
                                      initializer=self.kernel_initializer,
                                      trainable=True)

        self.bias = self.add_weight("bias",
                                    shape=(self.filters,),
                                    initializer=self.bias_initializer,
                                    trainable=True)

        mask = np.ones(self.kernel.shape, dtype=np.float32)


        if kernel_d % 2 != 0: 
            center_d = kernel_d // 2
        else:
            center_d = (kernel_d - 1) // 2

        if kernel_h % 2 != 0: 
            center_h = kernel_h // 2
        else:
            center_h = (kernel_h - 1) // 2

        if kernel_w % 2 != 0: 
            center_w = kernel_w // 2
        else:
            center_w = (kernel_w - 1) // 2


        if self.mask_type == 'D':
            mask[center_d:, :, :, :, :] = 0.
        elif self.mask_type == 'V':
            mask[center_d, center_h:, :, :, :] = 0.
            mask[center_d + 1:, :, :, :, :] = 0.
        else:
            mask[center_d, center_h, center_w + (self.mask_type == 'B'):, :, :] = 0.
            mask[center_d + 1:, :, :, :, :] = 0.
            mask[:, center_h + 1:, :, :, :] = 0.

        self.mask = tf.constant(mask, dtype=tf.float32, name='mask')

    def call(self, input):
        masked_kernel = tf.math.multiply(self.mask, self.kernel)
        x = tf.nn.conv3d(input, masked_kernel, strides=[1, self.strides, self.strides, self.strides, 1], padding=self.padding)
        x = tf.nn.bias_add(x, self.bias)
        return x

In [0]:
def build_test_stack_3d(mask_type, input_shape=(5, 5, 5, 1), kernel_size=(3, 3, 3)):
  inputs = tf.keras.layers.Input(shape=input_shape)
  
  x = MaskedConv3D(mask_type=mask_type,
                   filters=1,
                   kernel_size=kernel_size, 
                   padding='same',
                   kernel_initializer='ones', 
                   bias_initializer='zeros')(inputs)

  stack = tf.keras.Model(inputs=inputs, outputs=x)
  stack.compile(optimizer='adam', loss='mse')
  return stack

### Tests with kernel_size 3

In [16]:
depth_stack = build_test_stack_3d('D')
val = depth_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Depth 1
[[4. 6. 6. 6. 4.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [4. 6. 6. 6. 4.]]
Depth 2
[[4. 6. 6. 6. 4.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [4. 6. 6. 6. 4.]]
Depth 3
[[4. 6. 6. 6. 4.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [4. 6. 6. 6. 4.]]
Depth 4
[[4. 6. 6. 6. 4.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]
 [4. 6. 6. 6. 4.]]


In [17]:
vertical_stack = build_test_stack_3d('V', kernel_size=(1, 3, 3))
val = vertical_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[0. 0. 0. 0. 0.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]]
Depth 1
[[0. 0. 0. 0. 0.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]]
Depth 2
[[0. 0. 0. 0. 0.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]]
Depth 3
[[0. 0. 0. 0. 0.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]]
Depth 4
[[0. 0. 0. 0. 0.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]
 [2. 3. 3. 3. 2.]]


In [18]:
horizontal_A_stack = build_test_stack_3d('A', kernel_size=(1, 1, 3))
val = horizontal_A_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 1
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 2
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 3
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 4
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]


In [19]:
horizontal_B_stack = build_test_stack_3d('B', kernel_size=(1, 1, 3))
val = horizontal_B_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 1
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 2
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 3
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 4
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]


### Tests with kernel_size 4

In [24]:
depth_stack = build_test_stack_3d('D', kernel_size=(4, 4, 4))
val = depth_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
Depth 1
[[ 9. 12. 12.  9.  6.]
 [12. 16. 16. 12.  8.]
 [12. 16. 16. 12.  8.]
 [ 9. 12. 12.  9.  6.]
 [ 6.  8.  8.  6.  4.]]
Depth 2
[[ 9. 12. 12.  9.  6.]
 [12. 16. 16. 12.  8.]
 [12. 16. 16. 12.  8.]
 [ 9. 12. 12.  9.  6.]
 [ 6.  8.  8.  6.  4.]]
Depth 3
[[ 9. 12. 12.  9.  6.]
 [12. 16. 16. 12.  8.]
 [12. 16. 16. 12.  8.]
 [ 9. 12. 12.  9.  6.]
 [ 6.  8.  8.  6.  4.]]
Depth 4
[[ 9. 12. 12.  9.  6.]
 [12. 16. 16. 12.  8.]
 [12. 16. 16. 12.  8.]
 [ 9. 12. 12.  9.  6.]
 [ 6.  8.  8.  6.  4.]]


In [25]:
vertical_stack = build_test_stack_3d('V', kernel_size=(1, 4, 4))
val = vertical_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[0. 0. 0. 0. 0.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]]
Depth 1
[[0. 0. 0. 0. 0.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]]
Depth 2
[[0. 0. 0. 0. 0.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]]
Depth 3
[[0. 0. 0. 0. 0.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]]
Depth 4
[[0. 0. 0. 0. 0.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]
 [3. 4. 4. 3. 2.]]


In [26]:
horizontal_A_stack = build_test_stack_3d('A', kernel_size=(1, 1, 4))
val = horizontal_A_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 1
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 2
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 3
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]
Depth 4
[[0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1.]]


In [27]:
horizontal_B_stack = build_test_stack_3d('B', kernel_size=(1, 1, 4))
val = horizontal_B_stack.predict(test_ones_3d)
print_test_3d(val)

Depth 0
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 1
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 2
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 3
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]
Depth 4
[[1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]
 [1. 2. 2. 2. 2.]]


## Creating 2D cropping solution

In [0]:
import math

class VerticalCroppedConv2d(tf.keras.Model):
    def __init__(self,
                 filters,
                 kernel_size,
                 kernel_initializer, 
                 bias_initializer):
        super(VerticalCroppedConv2d, self).__init__(name='')

        if isinstance(kernel_size, int):
            kernel_size = (kernel_size, kernel_size)

        kernel_h, kernel_w = kernel_size

        self.padding = keras.layers.ZeroPadding2D(padding=((kernel_h-1, 0),(int((kernel_w-1)/2),int((kernel_w-1)/2))))

        self.conv = keras.layers.Conv2D(filters=filters,
                                        kernel_size=kernel_size,
                                        strides=1,
                                        padding='valid',
                                        kernel_initializer=kernel_initializer, 
                                        bias_initializer=bias_initializer)

    def call(self, input_value):

        x = self.padding(input_value)
        x = self.conv(x)
        out = self.cropping(x)

        return out



Example step by step

In [67]:
kernel_h = 2
kernel_w = 3

kernel_size = (kernel_h, kernel_w)

padding = keras.layers.ZeroPadding2D(padding=((kernel_h-1, 0),(int((kernel_w-1)/2),int((kernel_w-1)/2))))

res = padding(test_ones_2d)
print(res.numpy().squeeze())

conv = keras.layers.Conv2D(filters=1,
                                        kernel_size=kernel_size,
                                        strides=1,
                                        padding='valid',
                                        kernel_initializer='ones', 
                                        bias_initializer='zeros')

res2 = conv(res)
print(res2.numpy().squeeze())






If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

[[0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0.]
 [0. 1. 1. 1. 1. 1. 0.]]
[[2. 3. 3. 3. 2.]
 [4. 6. 6. 6. 4.]
 [4. 6. 6. 6. 4.]
 [4. 6. 6. 6. 4.]
 [4. 6. 6. 6. 4.]]


In [64]:
conv.weights[0].numpy().squeeze()

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [0]:
def build_test_croppedv_stack_2d(input_shape=(5, 5, 1), kernel_size=3):
  inputs = tf.keras.layers.Input(shape=input_shape)
  
  x = VerticalCroppedConv2d(
                   filters=1,
                   kernel_size=kernel_size, 
                   kernel_initializer='ones', 
                   bias_initializer='zeros')(inputs)

  stack = tf.keras.Model(inputs=inputs, outputs=x)
  stack.compile(optimizer='adam', loss='mse')
  return stack

###Tests with kernel_size 3

In [59]:
vertical_stack = build_test_croppedv_stack_2d()
val = vertical_stack.predict(test_ones_2d)
print(val[0,:,:,0].squeeze())

[[2. 3. 3. 3. 2.]
 [4. 6. 6. 6. 4.]
 [6. 9. 9. 9. 6.]
 [6. 9. 9. 9. 6.]]


REFERENCES

https://www.youtube.com/watch?v=1BURwCCYNEI